<a href="https://colab.research.google.com/github/PythonBeginners/Meetup034_StartML/blob/main/" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 参考URL
https://qiita.com/fukuit/items/215ef75113d97560e599

#### PyTorchのインストールについて
- Google Colabにはデフォルトでpytorchが入っているので、pipで入れる必要はありません

In [ ]:
# !pip install -U torch torchvision

#### ライブラリのimport

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

print(torch.__version__)

In [ ]:
# GPU(CUDA)が使えるときはGPUを使う
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

#### 標準化
[transforms.Compose()](https://pytorch.org/docs/stable/torchvision/transforms.html)によって、MNISTの各ピクセル値を正規分布になるように変換します。
以下は公式のリファレンスより抜粋

```
Normalize a tensor image with mean and standard deviation. Given mean: (M1,...,Mn) and std: (S1,..,Sn) for n channels, this transform will normalize each channel of the input torch.*Tensor i.e. output[channel] = (input[channel] - mean[channel]) / std[channel]
```

以下のコードではmean=0.5、std=0.5となるように変換するフィルタを定義しています。

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, ), (0.5, ))])

#### データのダウンロード

In [ ]:
trainset = torchvision.datasets.MNIST(root='./data', 
                                        train=True,
                                        download=True,
                                        transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,
                                            batch_size=100,
                                            shuffle=True,
                                            num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', 
                                        train=False, 
                                        download=True, 
                                        transform=transform)
testloader = torch.utils.data.DataLoader(testset, 
                                            batch_size=100,
                                            shuffle=False, 
                                            num_workers=2)

classes = tuple(np.linspace(0, 9, 10, dtype=np.uint8))

#### NeuralNetの定義

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3) # 28x28x32 -> 26x26x32
        self.conv2 = nn.Conv2d(32, 64, 3) # 26x26x64 -> 24x24x64 
        self.pool = nn.MaxPool2d(2, 2) # 24x24x64 -> 12x12x64
        self.dropout1 = nn.Dropout2d()
        self.fc1 = nn.Linear(12 * 12 * 64, 128)
        self.dropout2 = nn.Dropout2d()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout1(x)
        x = x.view(-1, 12 * 12 * 64)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

In [ ]:
net = Net().to(device)

#### loss関数と最適化の設定をする

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
epochs = 5
train_loss_list = []
train_acc_list = []

for epoch in range(epochs):
    running_loss = 0.0
    train_loss = 0
    train_acc = 0
    for i, (inputs, labels) in enumerate(trainloader, 0):

        # GPUが使える時はGPUにデータを送る
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += (outputs.max(1)[1] == labels).sum().item()


        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:
            print('[{:d}, {:5d}] loss: {:.3f}'
                    .format(epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

    avg_train_loss = train_loss / len(trainloader.dataset)
    avg_train_acc = train_acc / len(trainloader.dataset)
    train_loss_list.append(avg_train_loss)
    train_acc_list.append(avg_train_acc)

print('Finished Training')

In [ ]:
import 

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for (inputs, labels) in testloader:

        # GPUが使える時はGPUにデータを送る
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy: {:.2f} %%'.format(100 * float(correct/total)))

# やってみよう！
- [x] 画像を可視化してみよう！
- [x] 当てられている画像と当てられていない画像をそれぞれ可視化してみよう！
    - これはテストデータでやるのが尤もっぽい
- [x] precision、recall、f1-scoreを出してみよう！
    - classification_report使ったけどﾖｼ!
    - presisionは適合率。正と予測したデータのうち，実際に正であるものの割合
    - recallは再現率。実際に正であるもののうち，正であると予測されたものの割合。
    - f1は適合率と再現率の調和平均。これは1に近いほど良い精度といえる。
    - [参考](https://qiita.com/FukuharaYohei/items/be89a99c53586fa4e2e4)
- [x] 学習曲線を出してみよう！
    - [参考](https://qiita.com/makaishi2/items/3676d216fe9b34b63430)
- [ ] validationスコア、testスコアを出してみよう！
- [x] cross-validationしてみよう！
    - いまの状態だとしてみただけの状態になっている。
    - 本来はこれでデータに偏りがないことを調査したりする。
    - データに偏りがある場合、つまり学習に偏りがあることになり、正常な学習、精度とは言えなくなってしまう
    - 今回の例だと全てにおいて精度が良かったので大丈夫
- [ ] あまりにも汚い手書き文字は取り除いて学習してみよう！

In [ ]:
testloader


In [ ]:
for i, data in enumerate(testloader):
    x, y = data

In [ ]:
test_loader = torch.utils.data.DataLoader(testset,batch_size=20,shuffle=False)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report


In [ ]:
def image_show(data_loader,n, is_correct: bool):

  #Augmentationした画像データを読み込む
  tmp = iter(data_loader)
  images,labels = next(tmp)
  outputs = net(images.to(device))
  _, predicted = torch.max(outputs.data, 1)

  #n枚の画像を1枚ずつ取り出し、表示する
  for i in range(n):
    ans = predicted[i] == labels[i]
    if ans == is_correct:
      #画像をtensorからnumpyに変換
      images = images.numpy()
      print("予測値：", predicted[i])
      print("正解値：", labels[i])
      print(predicted[i] == labels[i])

      image = np.transpose(images[i],[1,2,0])
      plt.imshow(image)
      plt.show()

  print(classification_report(labels, predicted.detach().cpu().numpy()))
    

image_show(test_loader,20, False)

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(train_acc_list,label='adam', lw=3, c='b')
plt.title('accuracy')
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.legend(fontsize=14)
plt.show()

In [ ]:
# 学習曲線 (損失関数値)
plt.figure(figsize=(8,6))
plt.plot(train_loss_list,label='adam', lw=3, c='b')
plt.title('loss')
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.legend(fontsize=14)
plt.show()

In [ ]:
# 交差検証(cross varidation)を行ってみる

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from torch.utils.data.dataset import Subset


dataset = torchvision.datasets.MNIST(root='./data',download=True,transform=transform)

batch_size = 16
kf = KFold(n_splits=3)

nets, accs, losses = [], [], []


cv = 0
for _fold, (train_index, valid_index) in enumerate(kf.split(dataset)):
    train_dataset = Subset(dataset, train_index)
    train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
    valid_dataset   = Subset(dataset, valid_index)
    valid_dataloader = DataLoader(valid_dataset, batch_size, shuffle=False)

    classes = tuple(np.linspace(0, 9, 10, dtype=np.uint8))

    for epoch in range(epochs):
        running_loss = 0.0
        train_loss = 0
        train_acc = 0
        for i, (inputs, labels) in enumerate(train_dataloader, 0):

            # GPUが使える時はGPUにデータを送る
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_acc += (outputs.max(1)[1] == labels).sum().item()


            # print statistics
            running_loss += loss.item()
            if i % 100 == 99:
                print('[{:d}, {:5d}] loss: {:.3f}'
                        .format(epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

        avg_train_loss = train_loss / len(trainloader.dataset)
        avg_train_acc = train_acc / len(trainloader.dataset)
        train_loss_list.append(avg_train_loss)
        train_acc_list.append(avg_train_acc)

    print('Finished Training')
    correct = 0
    total = 0

    with torch.no_grad():
        for (inputs, labels) in testloader:

            # GPUが使える時はGPUにデータを送る
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy: {:.2f} %%'.format(100 * float(correct/total)))

    cv += train_loss / kf.n_splits
    nets.append(net)
    accs.append(correct/total)
    losses.append(loss)

print(accs)
print(losses)
